In [37]:
import ccxt
import pandas as pd

pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [38]:
# Initialize the exchange
binance = ccxt.binance()

# Define the symbols and timeframe
symbols = ['BTC/USDT', 'ETH/USDT', 'XRP/USDT', 'DOGE/USDT', 'SOL/USDT']
timeframe = '1h'  # 1-hour candles
limit = 1000 # Number of candles to fetch per request
start_date = '2024-01-01T00:00:00Z'

def fetch_data(symbol, timeframe, start_date, limit):
    since = binance.parse8601(start_date)
    symbol_ohlcv = []
    while True:
        ohlcv = binance.fetch_ohlcv(symbol, timeframe, since, limit)
        if len(ohlcv) == 0:
            break
        symbol_ohlcv.extend(ohlcv)
        since = ohlcv[-1][0] + 1
    return symbol_ohlcv

# Fetch the data
all_ohlcv = pd.DataFrame([], columns=['timestamp', 'symbol', 'open', 'high', 'low', 'close', 'volume'])

for symbol in symbols:
    symbol_ohlcv = fetch_data(symbol, timeframe, start_date, limit)
    df = pd.DataFrame(symbol_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['symbol'] = symbol
    all_ohlcv = pd.concat([all_ohlcv, df])
    print(f"Fetched {len(symbol_ohlcv)} candles for {symbol}")

/tmp/ipykernel_25422/410943163.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_ohlcv = pd.concat([all_ohlcv, df])


Fetched 13094 candles for BTC/USDT
Fetched 13094 candles for ETH/USDT
Fetched 13094 candles for XRP/USDT
Fetched 13094 candles for DOGE/USDT
Fetched 13094 candles for SOL/USDT


In [39]:
all_ohlcv['timestamp'] = pd.to_datetime(all_ohlcv['timestamp'], unit='ms')
all_ohlcv.set_index('timestamp', inplace=True)
all_ohlcv.sort_index(inplace=True)

print(f"Fetched {len(all_ohlcv)} candles")

Fetched 65470 candles


In [40]:
all_ohlcv.head(5)

,symbol,open,high,low,close,volume
timestamp,,,,,,
2024-01-01,BTC/USDT,42283.5800,42554.5700,42261.0200,42475.2300,1271.6811
2024-01-01,DOGE/USDT,0.0896,0.0900,0.0895,0.0898,17799677.0000
2024-01-01,XRP/USDT,0.6155,0.6172,0.6146,0.6162,14498728.0000
2024-01-01,ETH/USDT,2281.8700,2297.1800,2281.2700,2295.5100,10771.9183
2024-01-01,SOL/USDT,101.7200,102.7900,101.5600,101.9600,196680.9300


In [41]:
all_ohlcv.to_csv('data/ohlcv.csv.gz', compression='gzip')